# Process `copa` for translations

In [2]:
import csv
import os
import json
res_output_path = "_data/"
os.makedirs(res_output_path, exist_ok=True)

input_path = "/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/SuperGLUE/COPA/val.jsonl"
#input_path = "/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/paws_x/x-final/en/test_2k.tsv"
id_prefix = "/".join(input_path.split("/")[-5:])
items_to_export = 100

input_data = []
for line in open(input_path):
    item = json.loads(line.strip())
    input_data.append(item)
if items_to_export <= 0:
    items_to_export = len(input_data) 

output_path = res_output_path + "copa_en"+os.path.basename(input_path)+f"{items_to_export:04}.tsv"
print(input_data[0])

{'premise': 'The man turned on the faucet.', 'choice1': 'The toilet filled with water.', 'choice2': 'Water flowed from the spout.', 'question': 'effect', 'label': 1, 'idx': 0}


In [3]:
# 

new_items = []
for i, item in enumerate(input_data):
#     new_item = {
#         "id": f"{id_prefix}_{item['id']}",
#         "text": " ".join([item[x].strip() for x in ['sentence1', 
#                                                      'sentence2', 
#                                                      ]]),
#     }
    new_item = item
    new_item["id"] = f"{id_prefix}_{i}"
    new_items.append(new_item)
    #print(new_item)
     
print(f"Total items:{len(new_items)}")
new_items = new_items[:items_to_export]
print(f"Items to export:{len(new_items)}")

with open(output_path, mode="w") as f_out:
    writer = csv.DictWriter(f_out, new_item.keys(), delimiter='\t')
    writer.writeheader()
    writer.writerows(new_items)
    
print(f"Data exported to:\n{os.path.abspath(output_path)}")
        



Total items:100
Items to export:100
Data exported to:
/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/scripts/task_translation/_data/copa_enval.jsonl0100.tsv


# Convert translated tsv file to jsonl

In [4]:
import csv
import json
tsv_file = "/private/home/tbmihaylov/data/xlmg/few_shot/xcopa/human_translation/ru/val.ru.tsv"
json_out_file = tsv_file.replace(".tsv", ".jsonl")
with open(json_out_file, mode="w") as f_out:
    columns = []
    delim = "\t"
    for ii, line in enumerate(open(tsv_file)):
        if ii == 0:
            columns = line.strip().split(delim)
            continue
        vals = line.strip().split(delim)
        if len(vals) != len(columns):
            raise Exception(f"Item {ii} in {input_path}: values count {len(vals)} does not match columns count {len(columns)}\n {vals}{columns} ")

        item = {kv[0]:kv[1] for kv in zip(columns, vals)}
        if item["label"] not in {"0", "1", 0, 1}:
            print(item)
            raise Exception(f"Item {ii} in {input_path} has unknown label '{item['label']}'")
        item["label"] = int(item["label"])
        f_out.write(json.dumps(item, ensure_ascii=False).strip())
        f_out.write("\n")
print(json_out_file)

/private/home/tbmihaylov/data/xlmg/few_shot/xcopa/human_translation/ru/val.ru.jsonl
